<a href="https://www.kaggle.com/code/icewallowcomeyuff/ml-video-conferencing-analysis?scriptVersionId=215658005" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Machine Learning Based Traffic Analysis of Video Conferencing Services in rural communities - Main

# 📚 | Import Libraries

In [ ]:
import keras_cv
import keras
from keras import ops
import tensorflow as tf

import cv2
import pandas as pd
import numpy as np
from glob import glob
from tqdm.notebook import tqdm
import joblib
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler


import matplotlib.pyplot as plt

## Library Versions

In [ ]:
print("TensorFlow:", tf.__version__)
print("Keras:", keras.__version__)
print("KerasCV:", keras_cv.__version__)

# 💾 | Data Preparation

## Initial Inspection

### The Zoom480pRaw Dataset contains the following QoS metrics :

#### 1. Local time : The timestamp indicating when the data was captured.
#### 2. Download (Rx) bandwidth (bps) : The bandwidth available for downloading data, measured in bits per second (bps).
#### 3. Upload (Tx) bandwidth (bps) : The bandwidth available for uploading data, measured in bits per second (bps).
#### 4. Rx packet loss (percent) : Percentage of packets lost during download.
#### 5. Tx packet loss (percent) : Percentage of packets lost during upload.
#### 6. RTT (ping) (ms : Round Trip Time (RTT) measures the time (in milliseconds) for a signal to travel to the server and back.
#### 7. Rx instant jitter (ms) : Instantaneous jitter during download, measured in millisecond
#### 8. Tx instant jitter (ms) : Instantaneous jitter during upload, measured in milliseconds.
#### 9. Rx RFC3550 jitter (ms) : Jitter measured according to the RFC3550 standard for downloading packets.
#### 10.Tx RFC3550 jitter (ms) : Jitter measured according to the RFC3550 standard for uploading packets
#### 11.Rx packet loss burst length (packets) : The number of consecutive packets lost during download.
#### 12.Tx packet loss burst length (packets) : The number of consecutive packets lost during upload.


#### 13.Download bandwidth &
#### 14.Upload Bandwidth : These columns represent textual descriptions of bandwidth values in a human-readable format, such as 2.08 bps/4.00 bps.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np

# Load the dataset
file_path = '/kaggle/input/zoom480p/zoom480pRaw.csv'  # Update this path based on your Kaggle dataset location
data = pd.read_csv(file_path)

# Display the first few rows of the dataset
print("First few rows of the dataset:")
display(data.head())

# Display basic information about the dataset
print("\nDataset Info:")
data.info()

# Check for missing values
print("\nMissing Values per Column:")
print(data.isnull().sum())

# Display summary statistics
print("\nSummary Statistics:")
display(data.describe())


## Redundancy Checking between Rx and Tx metrics.


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Calculate correlations between Rx and Tx metrics
rx_tx_correlations = data[[
    'Rx packet loss (percent)', 'Tx packet loss (percent)',
    'Rx instant jitter (ms)', 'Tx instant jitter (ms)'
]].corr()

# Display the correlation matrix
print("\nCorrelation Matrix for Rx and Tx Metrics:")
print(rx_tx_correlations)

# Visualize the correlation matrix
plt.figure(figsize=(8, 6))
sns.heatmap(rx_tx_correlations, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Between Rx and Tx Metrics')
plt.show()


### The low correlation values indicate that Rx and Tx metrics do not show significant redundancy. This suggests that both metrics capture different aspects of the network's QoS and should be retained. There is no strong redundancy between Rx and Tx jitter or packet loss. Hence, both sets of metrics can provide unique value for analyzing the dataset.

## Redundancy checking between RFC3550 jitter (ms) & Instant Jitter (ms)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Select jitter columns for correlation analysis
jitter_columns = [
    'Rx instant jitter (ms)', 'Tx instant jitter (ms)', 
    'Rx RFC3550 jitter (ms)', 'Tx RFC3550 jitter (ms)'
]

# Calculate correlation matrix
jitter_correlation = data[jitter_columns].corr()

# Display the correlation matrix
print("Correlation Matrix for Jitter Metrics:")
print(jitter_correlation)

# Visualize the correlation matrix using a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(jitter_correlation, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Between Instant Jitter and RFC3550 Jitter")
plt.show()


### We can see high values (e.g., 0.8 or above) between:

### * Rx instant jitter (ms) and Rx RFC3550 jitter (ms).
### * Tx instant jitter (ms) and Tx RFC3550 jitter (ms).

### Since instant jitter is more intuitive and directly reflects real-time conditions, keeping it and dropping RFC3550 jitter simplifies the model without losing much information.

## Cleaning & Extracting Necessary Features

### Column selection retained the most relevant features, reducing unnecessary noise in the dataset.

### Columns Dropped:

### 1. Download bandwidth: Redundant, as Download (Rx) bandwidth (bps) provides the same information in numeric format.
### 2. Upload bandwidth:   Redundant, as Upload (Tx) bandwidth (bps) provides the same information in numeric format.

### 3. Rx RFC3550 jitter (ms)
### 4. Tx RFC3550 jitter (ms) 

### The dataset already includes Rx instant jitter (ms) and Tx instant jitter (ms), which capture jitter values for incoming and outgoing packets.The RFC3550 jitter columns represent a standardized method to compute jitter, but in practice, they are often highly correlated with the "instant jitter" columns.

### 5. Rx packet loss burst length (packets)
### 6. Tx packet loss burst length (packets)

### These columns measure the length of consecutive packets lost during download (Rx) or upload (Tx).
### While they provide detailed insights into packet loss events, their impact is already captured by Rx packet loss (percent) and Tx packet loss (percent): For example, if 5 packets are lost consecutively, it would already contribute to the total packet loss percentage.
### We chose Packet Loss Percent over the Number of Packets Dropped because it is normalized, allowing for easy comparison across datasets and services, regardless of the total packet count. Additionally, it is more directly correlated with QoE and easier for machine learning models to interpret and scale.

In [ ]:
# Select necessary numeric columns for analysis
columns_to_keep = [
    "Local time", 
    "Download (Rx) bandwidth (bps)", 
    "Upload (Tx) bandwidth (bps)",
    "Rx packet loss (percent)", 
    "Tx packet loss (percent)", 
    "RTT (ping) (ms)",
    "Rx instant jitter (ms)", 
    "Tx instant jitter (ms)"
]

# Subset the dataset
data = data[columns_to_keep]

# Display the cleaned subset
print("Selected Columns:")
display(data.head())

# Check for any missing or invalid values in the selected columns
print("\nMissing Values After Column Selection:")
print(data.isnull().sum())


## Mean Jitter & Jitter Difference

###  **Mean Jitter**

* ### QoE is often affected by the overall jitter in the network, not just Rx (receive) or Tx (transmit) jitter individually.
* ### Mean Jitter provides a single value summarizing the average network instability, representing the combined impact of incoming and outgoing jitter.

### **Jitter Difference**

* ### When there’s a significant difference between Rx and Tx jitter, it often indicates imbalanced network conditions (e.g., poor uplink but stable downlink).
* ### A large Jitter Difference value helps pinpoint whether the bottleneck is on the upload (Tx) or download (Rx) side of the network.


In [ ]:
data['Mean Jitter'] = (data['Rx instant jitter (ms)'] + data['Tx instant jitter (ms)']) / 2

data['Jitter Difference'] = data['Rx instant jitter (ms)'] - data['Tx instant jitter (ms)']


## Unit Consistency

###  Ensures all numeric metrics are in comparable units. bps (bandwidth) was converted to Mbps for easier interpretation.

In [ ]:
# Convert bandwidth metrics from bps to Mbps
data['Download (Rx) bandwidth (Mbps)'] = data['Download (Rx) bandwidth (bps)'] / 1e6
data['Upload (Tx) bandwidth (Mbps)'] = data['Upload (Tx) bandwidth (bps)'] / 1e6

# Drop the original bandwidth columns (in bps) to avoid confusion
data.drop(columns=['Download (Rx) bandwidth (bps)', 'Upload (Tx) bandwidth (bps)'], inplace=True)

# Display the updated dataset
print("\nDataset After Unit Conversion:")
display(data.head())


## Standardizing the Timestamp

### Timestamps in the raw dataset might be in string format, making it difficult to perform time-based calculations or analysis. Standardization converts them to a machine-readable datetime format.

In [ ]:
# Convert "Local time" to datetime format
data['Local time'] = pd.to_datetime(data['Local time'], errors='coerce')

# Check for invalid timestamps
print("\nRows with Invalid Timestamps:")
print(data[data['Local time'].isnull()])

# Remove rows with invalid timestamps
data = data.dropna(subset=['Local time'])

# Sort the data by timestamp (if not already sorted)
data = data.sort_values(by='Local time')

# Display the standardized timestamps
print("\nData After Standardizing Timestamps:")
display(data.head())

## Adding Elapsed Time Feature

### To introduce a temporal dimension, capturing how QoS metrics change over time. Helps identify whether network conditions improve or degrade over time.

In [ ]:
# Calculate elapsed time in seconds from the first timestamp
data['Elapsed Time'] = (data['Local time'] - data['Local time'].iloc[0]).dt.total_seconds()

# Display the dataset with the new "Elapsed Time" column
print("\nDataset with Elapsed Time:")
display(data.head(100))

## Handling Missing & Invalid Data

In [ ]:
# Check for missing values in numeric columns
numeric_columns = data.select_dtypes(include=[np.number]).columns
print("\nMissing Values in Numeric Columns Before Imputation:")
print(data[numeric_columns].isnull().sum())

# Impute missing values with the mean of each column
data[numeric_columns] = data[numeric_columns].fillna(data[numeric_columns].mean())

# Verify missing values are handled
print("\nMissing Values After Imputation:")
print(data[numeric_columns].isnull().sum())


## Saving the current prepared data

In [ ]:
print("\nDataset after anaylsis & preparation:")
display(data.head(100))

# After handling missing and invalid data
data['Service'] = 'Zoom'


# Save the cleaned and preprocessed data for downstream tasks
output_path = './zoom480p_preprocessed.csv'
data.to_csv(output_path, index=False)

print("\nPreprocessed dataset saved as 'zoom480p_preprocessed.csv'.")


# 📺 | Services Comparison (Google Meet , Microsoft Teams, Zoom)

## Loading & Inspecting Pre-processed Datasets

In [ ]:
# Load preprocessed datasets
gmeet_data = pd.read_csv('/kaggle/input/preprocessed-datasets/gmeet480p_preprocessed.csv')
teams_data = pd.read_csv('/kaggle/input/preprocessed-datasets/teams480p_preprocessed.csv')
zoom_data = pd.read_csv('/kaggle/input/preprocessed-datasets/zoom480p_preprocessed.csv')

# Display dataset info
print("Gmeet Dataset Info:")
print(gmeet_data.info(), "\n")
print("Teams Dataset Info:")
print(teams_data.info(), "\n")
print("Zoom Dataset Info:")
print(zoom_data.info(), "\n")


## Comparing & Visualizing Key QoS Metrics

### This code is used to compare and visualize the distributions of key QoS metrics.

In [ ]:
metrics = ['Rx packet loss (percent)', 'Tx packet loss (percent)', 
           'RTT (ping) (ms)', 'Rx instant jitter (ms)', 
           'Download (Rx) bandwidth (Mbps)']

# Visualize each metric for all services
datasets = {'Gmeet': gmeet_data, 'Teams': teams_data, 'Zoom': zoom_data}
for metric in metrics:
    plt.figure(figsize=(10, 6))
    # Combine datasets for the current metric with a 'Service' column
    combined_data = pd.concat([
        data[[metric]].assign(Service=service) for service, data in datasets.items()
    ], ignore_index=True)
    # Create boxplot
    sns.boxplot(data=combined_data, x='Service', y=metric, palette='Set3')
    plt.title(f'{metric} Distribution Across Services')
    plt.ylabel(metric)
    plt.xlabel('Service')
    plt.show()


### From the output:


* ### Gmeet shows consistent performance but with high outliers in RTT and jitter.
* ### Teams demonstrates variability in packet loss and stable bandwidth.
* ### Zoom maintains low outliers and balanced performance across metrics.


## Assessing Variability and Stability

### Variability of the dataset calculated by Standard Deviation.
###  Higher standard deviation indicates more variability in QoS metrics, which could lead to inconsistent QoE.

In [ ]:
# Compute standard deviation for QoS metrics
for service, data in datasets.items():
    print(f"Standard Deviation for {service}:")
    display(data[metrics].std())


### 1.  Google Meet


* ### High variability is observed in Tx packet loss (32.22%), RTT (155.07 ms), and Rx instant jitter (55.29 ms), indicating inconsistent performance.
* ### The bandwidth (0.49 Mbps) shows lower variability, suggesting more stable download speeds.\
* ### The high standard deviation in packet loss and RTT indicates QoS inconsistencies affecting user experience.

### 2. Microsoft Teams


* ### Low variability is observed in Tx packet loss (0.01%) and Rx instant jitter (11.09 ms), suggesting consistent performance in these metrics.
* ### RTT (13.85 ms) and Rx packet loss (0.78%) have moderate variability, indicating some instability.
* ### The low standard deviations indicate better overall stability compared to Gmeet and Zoom.

### 3. Zoom:


* ### Moderate variability is observed in Rx packet loss (0.65%), RTT (17.52 ms), and Rx instant jitter (11.65 ms), indicating relatively consistent performance.
* ### Bandwidth variability is low (0.19 Mbps), which helps maintain steady download speeds.
* ### The moderate standard deviations indicate a balance between stability and variability.



### Here, we Identify outliers in QoS metrics using the 1.5 × IQR rule for each service and metric.

### Outliers represent extreme values that deviate significantly from the typical range, potentially indicating extremely poor network performance with respect to our dataset.

In [ ]:
# Identify outliers for each metric using 1.5 * IQR rule
for service, data in datasets.items():
    print(f"Outliers for {service}:")
    for metric in metrics:
        q1 = data[metric].quantile(0.25)
        q3 = data[metric].quantile(0.75)
        iqr = q3 - q1
        outliers = data[(data[metric] < (q1 - 1.5 * iqr)) | (data[metric] > (q3 + 1.5 * iqr))]
        print(f"{metric}: {len(outliers)} outliers")



### 1. Gmeet: High outliers in packet loss, jitter, and bandwidth indicate unstable performance under extreme conditions.
### 2. Teams: Few outliers across metrics show consistent performance, with no bandwidth-related outliers.
### 3. Zoom: Significant outliers in Tx packet loss and moderate in jitter and RTT indicate occasional performance degradation.


### Insights
* ### Teams: Low variability and few outliers make it suitable for modeling stable QoE.
* ### Gmeet: High variability and outliers provide insights into extreme QoS cases.
* ### Zoom: Balanced variability and outliers make it a good mix of stable and unstable scenarios.


## Aggregating Performance Scores of Services (Gmeet, Teams, Zoom)

### The scores for each service (Gmeet, Teams, Zoom) were calculated to evaluate their performance based on key QoS metrics.

### Metrics Considered:


* ### Rx packet loss (percent): Measures packet loss during download (lower is better).
* ### Tx packet loss (percent): Measures packet loss during upload (lower is better).
* ### RTT (ping) (ms): Measures network latency (lower is better).### Rx instant jitter (ms): Measures variability in packet arrival times (lower is better).
* ### Download (Rx) bandwidth (Mbps): Measures network capacity (higher is better).


### Score Calculation Steps:


* ### Normalization: QoS metrics were normalized using Min-Max scaling to bring all values to a comparable range (0 to 1).
* ### Inversion: Normalized values were inverted (1 - normalized) so that better QoS values correspond to higher scores.
* ### Weighted Scoring: Scores were calculated using matrix multiplication with predefined weights for each metric.


### Weights were derived based on their relative impact on QoE, as identified in the research paper "Analysis of the Effect of QoS on Video Conferencing QoE".


1. ### Packet Loss (Rx/Tx): Most significant factors affecting QoE due to their direct impact on audio/video quality. Weight = 0.6 (0.3 each).
2. ### RTT (Latency): Affects responsiveness and synchronization during calls. Weight = 0.2.
3. ### Jitter: Causes frame drops and desynchronization. Weight = 0.1.
4. ### Bandwidth: Ensures sufficient capacity for high-quality video but is less critical under stable conditions. Weight = 0.1.


In [ ]:
metrics = ['Rx packet loss (percent)', 'Tx packet loss (percent)', 
           'RTT (ping) (ms)', 'Rx instant jitter (ms)', 
           'Download (Rx) bandwidth (Mbps)']

weights = {
    'Rx packet loss (percent)': 0.25,
    'Tx packet loss (percent)': 0.25,
    'RTT (ping) (ms)': 0.1,
    'Rx instant jitter (ms)': 0.3,
    'Download (Rx) bandwidth (Mbps)': 0.1
}

# Function to calculate scores
def calculate_scores(data, metrics, weights):
    # Normalize metrics
    scaler = MinMaxScaler()
    normalized = scaler.fit_transform(data[metrics])
    
    # Invert negative QoS metrics (lower is better)
    inverted = 1 - normalized  # Apply inversion for all metrics
    
    # Correct the inversion for bandwidth (higher is better)
    # Bandwidth is assumed to be the last metric in the list (adjust as necessary)
    inverted[:, metrics.index('Download (Rx) bandwidth (Mbps)')] = normalized[:, metrics.index('Download (Rx) bandwidth (Mbps)')]
    
    # Compute weighted score
    scores = inverted @ pd.Series(weights).values  # Matrix multiplication for scores
    return scores

# Re-run the score calculations with the updated function
gmeet_scores = calculate_scores(gmeet_data, metrics, weights)
teams_scores = calculate_scores(teams_data, metrics, weights)
zoom_scores = calculate_scores(zoom_data, metrics, weights)

# Add scores back to the datasets for analysis
gmeet_data['Score'] = gmeet_scores
teams_data['Score'] = teams_scores
zoom_data['Score'] = zoom_scores

# Calculate average scores for each service
average_scores = {
    'Gmeet': gmeet_data['Score'].mean(),
    'Teams': teams_data['Score'].mean(),
    'Zoom': zoom_data['Score'].mean()
}

print("Average Scores by Service:")
for service, score in average_scores.items():
    print(f"{service}: {score}")

# Plot average scores for each service
plt.figure(figsize=(10, 6))
plt.bar(average_scores.keys(), average_scores.values(), color=['skyblue', 'lightgreen', 'salmon'])
plt.title('Average Performance Scores by Service')
plt.ylabel('Average Score')
plt.xlabel('Service')
plt.ylim(0, max(average_scores.values()) + 0.1)  # Set y-limit slightly above the max score for better visualization
for i, score in enumerate(average_scores.values()):
    plt.text(i, score + 0.01, f"{score:.2f}", ha='center', fontsize=10)  # Add score labels on the bars
plt.show()



### The bar chart shows the average performance scores for Gmeet, Teams, and Zoom, highlighting Zoom as the best-performing service with a score of 0.90, followed by Teams (0.86) and Gmeet (0.83). This ranking reflects the combined weighted impact of key QoS metrics.

## Evaluating Dataset Suitability

### Dataset Diversity

### The code computes the diversity of QoS metrics (e.g., packet loss, RTT, jitter, bandwidth) for each service by calculating the range (difference between maximum and minimum values) of each metric.


* ### For each service, the diversity for each QoS metric is computed as: **Diversity = Max Value − Min Value**
* ### A larger range indicates greater diversity in the dataset for that metric.


### A dataset with higher diversity captures a broader range of network conditions, making it better suited for training a machine learning model that can generalize well. Low diversity may lead to models that overfit specific conditions and perform poorly in real-world scenarios.

In [ ]:
# Calculate diversity in QoS metrics for each service using range
diversity_data = {}
for service, data in datasets.items():
    diversity = data[metrics].max() - data[metrics].min()
    diversity_data[service] = diversity

# Convert to DataFrame for visualization
diversity_df = pd.DataFrame(diversity_data)

# Plot diversity for each metric
diversity_df.plot(kind='bar', figsize=(12, 8), title='Diversity in QoS Metrics Across Services')
plt.ylabel('Range of Values')
plt.xticks(rotation=45)
plt.legend(title="Service", loc='upper right')
plt.show()


###  Gmeet has the highest variability in critical metrics like RTT and Rx jitter. This indicates Gmeet’s dataset covers a broader range of conditions, making it valuable for training robust models that can generalize well.

### Dataset Size

### Analyzing the Dataset Size Across the 3 Services

In [ ]:
# Calculate dataset size for each service
dataset_sizes = {service: len(data) for service, data in datasets.items()}

# Plot dataset sizes
plt.figure(figsize=(10, 6))
plt.bar(dataset_sizes.keys(), dataset_sizes.values(), color='skyblue')
plt.title('Dataset Size Comparison')
plt.ylabel('Number of Rows')
plt.xlabel('Service')
plt.show()


### The bar chart compares dataset sizes, showing Zoom with the largest number of rows, followed by Gmeet, and then Teams. Larger dataset sizes, like Zoom's, ensure better model training and generalization by providing more data points for learning patterns.

### Distribution of Challenging/Poor QoS Conditons

### Thresholds Defined: Specific thresholds were established for key QoS metrics, such as jitter > 30 ms, packet loss > 1%, and RTT > 150 ms, based on the research paper "Analysis of the Effect of QoS on Video Conferencing QoE" 


### 1. For each metric and service, rows where the metric exceeds the threshold were identified and counted.
### 2. The count of challenging rows was divided by the total number of rows in the dataset, expressed as a percentage.




* ### This analysis helps determine which dataset contains a higher proportion of challenging conditions, making it more suitable for training models to handle poor network scenarios.
* ### Training models on datasets with a higher proportion of challenging conditions can improve their robustness and performance in real-world applications where such conditions are common.


In [ ]:
# Define thresholds for challenging conditions
thresholds = {
    'Rx instant jitter (ms)': 30,
    'Tx instant jitter (ms)': 30,
    'Rx packet loss (percent)': 1,
    'Tx packet loss (percent)': 1,
    'RTT (ping) (ms)': 150
}

# Calculate proportion of challenging conditions
challenging_data = {}
for service, data in datasets.items():
    proportions = {}
    for metric, threshold in thresholds.items():
        challenging_rows = len(data[data[metric] > threshold])
        proportions[metric] = challenging_rows / len(data) * 100  # Percentage
    challenging_data[service] = proportions

# Convert to DataFrame for visualization
challenging_df = pd.DataFrame(challenging_data)

# Plot challenging conditions for each metric
challenging_df.plot(kind='bar', figsize=(12, 8), title='Proportion of Challenging Conditions Across Services')
plt.ylabel('Percentage of Rows (%)')
plt.xticks(rotation=45)
plt.legend(title="Service", loc='upper right')
plt.show()



* ### Gmeet has the highest proportion of rows exceeding thresholds for Rx instant jitter, Tx instant jitter, and RTT, indicating frequent poor performance in these metrics.
* ### Zoom exhibits significant challenging conditions in Tx instant jitter but performs better in packet loss metrics compared to Gmeet.
* ### Teams shows the lowest proportion of challenging conditions across most metrics, particularly in jitter and packet loss, highlighting its stability under adverse conditions.


### Gmeet's dataset captures more challenging scenarios, making it valuable for training models to handle poor network conditions.


## Visualizing Dataset Suitability

### Here, we evaluate the overall suitability of the datasets (Gmeet, Teams, Zoom) for machine learning model training by combining three key factors: diversity, size, and challenging conditions.


### 1. Data Normalization: Z-Score Normalization is applied to ensure that all components are on the same scale, avoiding bias due to differing ranges in raw values.

* #### The raw values of the criteria (Diversity, Size, and Challenging Conditions) have different ranges and units.
* #### Z-Score Normalization standardizes these values by centering them around a mean of 0 with a standard deviation of 1, making them directly comparable.
* #### Min-Max normalization scales values between 0 and 1, but it is sensitive to outliers. If one dataset has extreme values in one criterion (e.g., high Diversity), it could dominate the normalized scores, skewing the final rankings.

### 2. Weights are assigned to each criterion:
   * * #### Diversity (50%): Captures network variability for better model generalization.
   * * #### Size (40%): Ensures sufficient data for robust training.
   * * #### Challenging Conditions (10%): Focuses on poor QoS scenarios for model robustness.
### 3.  A weighted final score is calculated for each dataset.


### Datasets are ranked based on their final weighted scores, with the highest-scoring dataset being the most suitable for further analysis & training 


In [ ]:
# Define the components of suitability
suitability_ranking = {
    'Diversity': diversity_df.mean(axis=0),  # Average diversity across all metrics
    'Size': pd.Series(dataset_sizes),       # Dataset size
    'Challenging Conditions': challenging_df.mean(axis=0)  # Average % of challenging conditions
}

# Combine into a DataFrame
suitability_df = pd.DataFrame(suitability_ranking)

# Ensure no missing values
suitability_df = suitability_df.fillna(0)

# Apply Z-Score Normalization
scaler = StandardScaler()
zscore_normalized_suitability = scaler.fit_transform(suitability_df)
suitability_df_normalized = pd.DataFrame(zscore_normalized_suitability, columns=suitability_df.columns, index=suitability_df.index)

# Define weights for each criterion
criteria_weights = {
    'Diversity': 0.5,
    'Size': 0.4,
    'Challenging Conditions': 0.1
}

# Calculate weighted final scores
suitability_df_normalized['Final Score'] = (
    suitability_df_normalized['Diversity'] * criteria_weights['Diversity'] +
    suitability_df_normalized['Size'] * criteria_weights['Size'] +
    suitability_df_normalized['Challenging Conditions'] * criteria_weights['Challenging Conditions']
)

# Sort by final score to determine the best dataset
suitability_df_normalized = suitability_df_normalized.sort_values('Final Score', ascending=False)

# Display rankings
print("Dataset Suitability Rankings (Weighted - Z-Score):")
display(suitability_df_normalized)

# Plot final scores
plt.figure(figsize=(10, 6))
plt.bar(suitability_df_normalized.index, suitability_df_normalized['Final Score'], color='skyblue')
plt.title('Final Suitability Score for Each Dataset (Z-Score Normalization)')
plt.ylabel('Weighted Final Score')
plt.xlabel('Service')
plt.show()


### Gmeet has the highest final score (1.04), indicating it is the most suitable dataset for model training & further analysis due to its high diversity and significant coverage of challenging conditions.
### Teams has the lowest score (-0.99), primarily due to its small dataset size and low diversity.

# 📈| Exploratory Data Analysis (EDA)(Google Meet)

## Inspecting the Dataset

In [ ]:
# Load the dataset
gmeet_data = pd.read_csv('/kaggle/input/preprocessed-datasets/gmeet480p_preprocessed.csv')

# Display basic information and statistics
print("Dataset Info:")
gmeet_data.info()

print("\nBasic Statistics:")
display(gmeet_data.describe())

# Check for missing values
print("\nMissing Values:")
display(gmeet_data.isnull().sum())


## Visualizing Feature Distributions

### Histograms and Kernel Density Plots

In [ ]:
# Plot histograms and KDE for QoS metrics
metrics = ['Rx packet loss (percent)', 'Tx packet loss (percent)', 
           'RTT (ping) (ms)', 'Rx instant jitter (ms)', 'Tx instant jitter (ms)', 
           'Download (Rx) bandwidth (Mbps)', 'Upload (Tx) bandwidth (Mbps)']

for metric in metrics:
    plt.figure(figsize=(10, 6))
    sns.histplot(gmeet_data[metric], kde=True, bins=30, color='skyblue')
    plt.title(f'{metric} - Distribution')
    plt.xlabel(metric)
    plt.ylabel('Frequency')
    plt.show()




### Key Points

### Upload (Tx) reliability is less stable than Download (Rx) , with frequent instances of higher packet loss.
### Similiarly, Tx instant jitter & Upload (Tx) Bandwidth is more unstable & spread compared to its Rx counterparts.
### This is in line with the notion of Tx conditions to be considered important in optimizing network conditions compared to the Rx. 

### Distributions of Mean Jitter & Jitter Difference

In [ ]:
# Visualize distributions of mean jitter and jitter difference
jitter_metrics = ['Mean Jitter', 'Jitter Difference']

for metric in jitter_metrics:
    plt.figure(figsize=(10, 6))
    sns.histplot(gmeet_data[metric], kde=True, bins=30, color='skyblue')
    plt.title(f'{metric} - Distribution')
    plt.xlabel(metric)
    plt.ylabel('Frequency')
    plt.show()

# Box plots
for metric in jitter_metrics:
    plt.figure(figsize=(10, 6))
    sns.boxplot(y=gmeet_data[metric], color='lightgreen')
    plt.title(f'{metric} - Box Plot')
    plt.ylabel(metric)
    plt.show()



* ### ITU-T Recommendation G.1010 specifies that jitter values exceeding 30 ms can cause noticeable quality degradation, with values above 50 ms considered poor for video conferencing.
* ### The histogram and box plots indicate that majority of the mean jitter values lie outside the acceptable values.
* ### Jitter Difference is calculated as Rx instant jitter - Tx instant jitter. Negative values indicate the transmission (Tx) side faces more jitter, while positive values show higher jitter on the reception (Rx) side.
* ### Significant negative values suggest the bottleneck is on the Tx side.


## Correlation Analysis

### Here, we calculate the correlation matrix for the QoS metrics in the dataset, which measures the linear relationship between each pair of metrics. Values range from -1 (strong negative correlation) to +1 (strong positive correlation).

In [ ]:
# Calculate correlation matrix
correlation_matrix = gmeet_data[metrics].corr()

# Plot heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Heatmap of QoS Metrics')
plt.show()



* ### Between Tx Packet Loss and Upload Bandwidth, a strong negative correlation (-0.70) exists, indicating that as upload bandwidth decreases, the percentage of Tx packet loss increases. This suggests poor upload bandwidth significantly affects transmission quality.
* ###  Rx instant jitter and Tx instant jitter do not strongly correlate with other metrics (close to 0), showing they may independently contribute to QoE issues.
* ### The correlation between RTT and Tx Packet Loss is moderate (0.48). This suggests that as transmission packet loss increases, network latency (RTT) also tends to increase.
* ### The correlation between Tx Instant Jitter and Upload Bandwidth is mild (0.41). This implies that unstable or lower upload bandwidth contributes to higher jitter, impacting the smooth delivery of video and audio during conferencing.


## Time-Series Trends

### Bandwidth over Time

### Here, we plot the download bandwidth (Download (Rx) bandwidth) over time, visualizing how bandwidth varies throughout the captured session.

In [ ]:
# Convert 'Local time' to datetime if available
if 'Local time' in gmeet_data.columns:
    gmeet_data['Local time'] = pd.to_datetime(gmeet_data['Local time'])

# Plot bandwidth over time
if 'Local time' in gmeet_data.columns:
    plt.figure(figsize=(12, 6))
    plt.plot(gmeet_data['Local time'], gmeet_data['Download (Rx) bandwidth (Mbps)'], color='blue', label='Bandwidth')
    plt.title('Download Bandwidth Over Time')
    plt.xlabel('Time')
    plt.ylabel('Bandwidth (Mbps)')
    plt.legend()
    plt.show()
else:
    print("Local time column not available for time-series plotting.")



* ### The plot shows noticeable fluctuations in download bandwidth over time, with several drops below 1 Mbps.
* ### Bandwidth drops below 2 Mbps, particularly during video conferencing, can degrade quality by causing buffering, pixelation, or delays.


### Jitter Over Time

### The code visualizes how the Rx instant jitter metric changes over time to assess network stability for received packets.

In [ ]:
# Plot jitter over time
if 'Local time' in gmeet_data.columns:
    plt.figure(figsize=(12, 6))
    plt.plot(gmeet_data['Local time'], gmeet_data['Rx instant jitter (ms)'], color='orange', label='Rx Jitter')
    plt.title('Rx Jitter Over Time')
    plt.xlabel('Time')
    plt.ylabel('Jitter (ms)')
    plt.legend()
    plt.show()
else:
    print("Local time column not available for time-series plotting.")



* ### The jitter plot reveals frequent fluctuations with some significant spikes exceeding 700 ms, indicating intermittent network instability.
* ### High jitter values, especially above the ITU-T G.1010 threshold of 30 ms for conversational applications, suggest potential disruptions in video quality, such as delays or audio-video desynchronization.


# 🎯| Deriving Target Variable 

##  Threshold-Based Target Derivation


### Here, we derive an initial target variable (`Initial_QoE`) based on ITU-T thresholds for QoS metrics. ie. ITU-T G.1010 & ITU-T G.114

### **Steps**:
1. ### **Define Thresholds**:
   - ### ITU-T Recommendations were used to define thresholds for key QoS metrics:
     - ### **Packet Loss (%)**: Good (<1%), Fair (1–3%), Poor (>3%).
     - ### **RTT (ms)**: Good (<150ms), Fair (150–300ms), Poor (>300ms).
     - ### **Jitter (ms)**: Good (<30ms), Fair (30–50ms), Poor (>50ms).
2. ### **Assign Individual Metric Labels**:
   - ### Each metric is labeled as "Good," "Fair," or "Poor" based on these thresholds.
3. ### **Aggregate Labels**:
   - ### Labels across all metrics are aggregated using a majority voting system to derive an overall QoE label (`Initial_QoE`).


### Define Thresholds and Apply Categorization

In [ ]:
import pandas as pd
import numpy as np

# Load the dataset
gmeet_data = pd.read_csv('/kaggle/input/preprocessed-datasets/gmeet480p_preprocessed.csv')

# Define ITU-T thresholds for QoS metrics
thresholds = {
    'Rx packet loss (percent)': [1, 3],  # Good (<1%), Fair (1-3%), Poor (>3%)
    'Tx packet loss (percent)': [1, 3],
    'RTT (ping) (ms)': [150, 300],      # Good (<150ms), Fair (150-300ms), Poor (>300ms)
    'Rx instant jitter (ms)': [30, 50], # Good (<30ms), Fair (30-50ms), Poor (>50ms)
    'Tx instant jitter (ms)': [30, 50]
}

# Function to assign thresholds
def assign_labels(row, metric, threshold):
    if row[metric] < threshold[0]:
        return 'Good'
    elif threshold[0] <= row[metric] <= threshold[1]:
        return 'Fair'
    else:
        return 'Poor'

# Apply thresholds to derive labels
for metric, threshold in thresholds.items():
    gmeet_data[f'{metric}_label'] = gmeet_data.apply(assign_labels, axis=1, metric=metric, threshold=threshold)

# Aggregate individual metric labels into an overall QoE label
def majority_vote(row):
    labels = [row[f'{metric}_label'] for metric in thresholds.keys()]
    return max(set(labels), key=labels.count)

gmeet_data['Initial_QoE'] = gmeet_data.apply(majority_vote, axis=1)


# View the dataset with categorized metrics
gmeet_data.head()

In [ ]:
# Check counts for threshold-based labels (Initial_QoE)
print("Counts for Threshold-Based Labels (Initial_QoE):")
threshold_counts = gmeet_data['Initial_QoE'].value_counts()
print(threshold_counts)


## Semi-Supervised Clustering

### Normalize Data and Initialize K-Means

### Here, we refine the target variable using K-Means clustering with initialization based on the threshold-derived labels (`Initial_QoE`).

### **Steps**:
### 1. - Standardized the features using `StandardScaler` to ensure equal contribution in clustering.
### 2. **Compute Initial Cluster Centers**:
   - ### Calculated median QoS values for each `Initial_QoE` category (Good, Fair, Poor) to initialize K-Means cluster centers.
### 3. **Apply K-Means**:
   - ### Clustering with 3 clusters (`Good`, `Fair`, `Poor`) using the initialized cluster centers.
### 4. **Map Clusters to QoE Labels**:
   - ### Mapped the numeric cluster labels to string equivalents (Good, Fair, Poor) for interpretability.
### 5. **Update Dataset**:
   - ### Added new columns:
     - ### `Cluster_QoE`: Numeric cluster labels.
     - ### `Cluster_QoE_mapped`: String-mapped QoE labels.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Select QoS metrics for clustering
qos_features = ['Rx packet loss (percent)', 'Tx packet loss (percent)', 
                'RTT (ping) (ms)', 'Rx instant jitter (ms)', 'Tx instant jitter (ms)']

# Standardize QoS features
scaler = StandardScaler()
qos_scaled = scaler.fit_transform(gmeet_data[qos_features])

# Compute initial cluster centers based on threshold-based labels
initial_centers = gmeet_data.groupby('Initial_QoE')[qos_features].mean()

# K-Means clustering with initialized cluster centers
kmeans = KMeans(n_clusters=3, init=initial_centers.values, n_init=1)
gmeet_data['Cluster_QoE'] = kmeans.fit_predict(qos_scaled)

# Map cluster labels to string equivalents
cluster_label_mapping = {0: 'Good', 1: 'Fair', 2: 'Poor'}  # Adjust based on your clustering results
gmeet_data['Cluster_QoE_mapped'] = gmeet_data['Cluster_QoE'].map(cluster_label_mapping)

gmeet_data.head()


### Counts of Threshold-Based Labels & Clustering-Based Labels

In [ ]:
# Check counts for threshold-based labels (Initial_QoE)
print("Counts for Threshold-Based Labels (Initial_QoE):")
threshold_counts = gmeet_data['Initial_QoE'].value_counts()
print(threshold_counts)

# Check counts for clustering-based labels (Cluster_QoE_mapped)
print("\nCounts for Clustering-Based Labels (Cluster_QoE_mapped):")
clustering_counts = gmeet_data['Cluster_QoE_mapped'].value_counts()
print(clustering_counts)


### No. Of Mismatches between both

In [ ]:
# Compare Initial_QoE and Cluster_QoE_mapped
mismatches = gmeet_data[gmeet_data['Initial_QoE'] != gmeet_data['Cluster_QoE_mapped']]
print(f"Number of mismatches: {len(mismatches)}")
print(mismatches[['Initial_QoE', 'Cluster_QoE_mapped']].head())


###  The similarity in counts between the threshold-based (Initial_QoE) and clustering-based (Cluster_QoE_mapped) labels but the low Cohen's Kappa Score indicates that the two methods are not assigning the same labels to individual data points consistently, even though their overall class distributions are comparable.

## Evaluate and Compare Methods

### Here, we :

* ### Encode Final_QoE (mapped to numeric values).
  
* ### The clustering-based labels (Cluster_QoE) are evaluated using:
    * ### Silhouette Score: Measures how well-defined the clusters are; closer to 1 is better.

    * ### Davies-Bouldin Index: Evaluates cluster separation; lower values are better.

    * ### Calinski-Harabasz Index: Measures compactness and separation; higher values are better.
 
* ### The Feature-Target Correlation calculates the relationship between QoS metrics (qos_features) and the encoded target variable (Final_QoE_numeric).



In [ ]:
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score, confusion_matrix, ConfusionMatrixDisplay

# Encode all QoE labels to numeric values
qoe_mapping = {'Good': 2, 'Fair': 1, 'Poor': 0}
gmeet_data['Initial_QoE_numeric'] = gmeet_data['Initial_QoE'].map(qoe_mapping)
gmeet_data['Cluster_QoE_numeric'] = gmeet_data['Cluster_QoE_mapped'].map(qoe_mapping)

# Correlation analysis for all labels
correlations = {
    'Initial_QoE': gmeet_data[qos_features + ['Initial_QoE_numeric']].corr()['Initial_QoE_numeric'],
    'Cluster_QoE': gmeet_data[qos_features + ['Cluster_QoE_numeric']].corr()['Cluster_QoE_numeric'],
}

print("Feature-Target Correlation:")
for label, correlation in correlations.items():
    print(f"\nCorrelation with {label}:")
    print(correlation)

# Clustering quality metrics for all labels
metrics = {}
for label, cluster_col in zip(
    ['Initial_QoE', 'Cluster_QoE'],
    ['Initial_QoE_numeric', 'Cluster_QoE']
):
    silhouette = silhouette_score(qos_scaled, gmeet_data[cluster_col])
    davies_bouldin = davies_bouldin_score(qos_scaled, gmeet_data[cluster_col])
    calinski_harabasz = calinski_harabasz_score(qos_scaled, gmeet_data[cluster_col])
    
    metrics[label] = {
        'Silhouette Score': silhouette,
        'Davies-Bouldin Index': davies_bouldin,
        'Calinski-Harabasz Index': calinski_harabasz
    }

print("\nClustering Quality Metrics:")
for label, metric_values in metrics.items():
    print(f"\nMetrics for {label}:")
    for metric_name, value in metric_values.items():
        print(f"{metric_name}: {value}")

gmeet_data.head()

### Threshold Based Labels

* ### The threshold-based labels capture some logical relationships (negative correlations), but these correlations are weak overall, which raises concerns about their predictive strength.
* ### Based on ITU-T standards, these labels align with domain knowledge and logical QoS relationships (negative correlations with impairments).
* ### Correlations with QoS features are weak, indicating limited predictive power.
* ### Clustering metrics reveal poorly separated clusters, which may hinder model performance.

### Clustering (K-Means) Based Labels

* ### Relies on clustering assumptions (e.g., distance-based metrics), which might not align with domain-specific QoE relationships.
* ### Higher clustering quality scores (Silhouette, Davies-Bouldin, and Calinski-Harabasz), indicating better-defined clusters.
* ### QoS features were standardized before clustering, so the clustering process doesn't explicitly account for the directional relationship between metrics and QoE.

##  Consolidating Results and Selecting the Best Method

### Using both Initial_QoE & Cluster_QoE as labels as seperate datasets.

In [ ]:
import pandas as pd

# Dataset with Threshold-Based Labels
dataset_threshold = gmeet_data.copy()
dataset_threshold['Target'] = gmeet_data['Initial_QoE']

# Drop other unwanted columns for Threshold-Based dataset
dataset_threshold = dataset_threshold.drop(
    columns=['Initial_QoE_numeric', 'Cluster_QoE', 'Cluster_QoE_mapped','Final_QoE', 'Final_QoE_numeric','Cluster_QoE_numeric','Initial_QoE'],
    errors='ignore'
)

# Save the Threshold-Based dataset
dataset_threshold.to_csv('/kaggle/working/gmeet_480p_threshold.csv', index=False)

# Dataset with Clustering-Based Labels
dataset_clustering = gmeet_data.copy()
dataset_clustering['Target'] = gmeet_data['Cluster_QoE_mapped']

# Drop other unwanted columns for Clustering-Based dataset
dataset_clustering = dataset_clustering.drop(
    columns=['Initial_QoE', 'Initial_QoE_numeric', 'Final_QoE', 'Final_QoE_numeric','Cluster_QoE','Cluster_QoE_mapped','Cluster_QoE_numeric'],
    errors='ignore'
)

# Save the Clustering-Based dataset
dataset_clustering.to_csv('/kaggle/working/gmeet_480p_cluster.csv', index=False)


### Finalizing & Saving Dataset

In [ ]:
import pandas as pd

# Read the Threshold-Based dataset
dataset_threshold = pd.read_csv('/kaggle/input/gmeet-label/gmeet_480p_threshold.csv')
print("Threshold-Based Dataset Sample:")
print(dataset_threshold.head())

# Display target counts for Threshold-Based dataset
print("\nClass Distribution in Threshold-Based Dataset:")
print(dataset_threshold['Target'].value_counts())

# Read the Clustering-Based dataset
dataset_clustering = pd.read_csv('/kaggle/input/gmeet-label/gmeet_480p_cluster.csv')
print("\nClustering-Based Dataset Sample:")
print(dataset_clustering.head())

# Display target counts for Clustering-Based dataset
print("\nClass Distribution in Clustering-Based Dataset:")
print(dataset_clustering['Target'].value_counts())


# ⚙️| Feature Engineering

## Initial Split (Train-Validation-Test Split)

In [ ]:
from sklearn.model_selection import train_test_split

# Load the dataset
gmeet_data = pd.read_csv('/kaggle/input/gmeet-label/gmeet_480p_threshold.csv')

# Split the dataset into train, validation, and test sets (stratified)
train_data, test_data = train_test_split(
    gmeet_data, test_size=0.2, random_state=42, stratify=gmeet_data['Target']
)
train_data, val_data = train_test_split(
    train_data, test_size=0.25, random_state=42, stratify=train_data['Target']
)  # 20% of original data will be validation

# Confirm distributions
print("Training Set Class Distribution:\n", train_data['Target'].value_counts())
print("\nValidation Set Class Distribution:\n", val_data['Target'].value_counts())
print("\nTesting Set Class Distribution:\n", test_data['Target'].value_counts())


## Outlier Detection and Handling

In [ ]:
import matplotlib.pyplot as plt

# Function to detect and handle outliers on training data only
def calculate_iqr_bounds(df, feature):
    """Calculate IQR bounds for outlier detection."""
    Q1 = df[feature].quantile(0.25)
    Q3 = df[feature].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return lower_bound, upper_bound

def handle_outliers(df, feature, lower_bound, upper_bound):
    """Replace outliers with the median value."""
    median = df[feature].median()
    df.loc[(df[feature] < lower_bound) | (df[feature] > upper_bound), feature] = median

# Handle outliers for training set and propagate bounds to validation and test sets
for feature in ['Rx packet loss (percent)', 'Tx packet loss (percent)', 
                'RTT (ping) (ms)', 'Rx instant jitter (ms)', 'Tx instant jitter (ms)']:
    # Calculate IQR bounds on the training set
    lower_bound, upper_bound = calculate_iqr_bounds(train_data, feature)
    
    # Handle outliers in the training set
    handle_outliers(train_data, feature, lower_bound, upper_bound)
    
    # Apply the same bounds to validation and test sets
    median = train_data[feature].median()
    val_data.loc[(val_data[feature] < lower_bound) | (val_data[feature] > upper_bound), feature] = median
    test_data.loc[(test_data[feature] < lower_bound) | (test_data[feature] > upper_bound), feature] = median

# Verify the data after outlier handling
print("Training Data Summary:\n", train_data.describe())
print("\nValidation Data Summary:\n", val_data.describe())
print("\nTest Data Summary:\n", test_data.describe())


### Extreme values for all numerical features have been mitigated, as evidenced by the statistics:
### Features like Rx packet loss (percent) and Tx instant jitter (ms) now have reduced ranges, with their max values appearing more consistent with typical QoS conditions.

### The mean and std (standard deviation) values for all features suggest a more normalized and less skewed dataset. This implies that the replacement of outliers (e.g., with median values) has successfully reduced distortions in the data.

## Feature Importance and Selection

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import mutual_info_classif

# Feature Importance using Random Forest (on training set)
qos_features = [
    'Rx packet loss (percent)', 'Tx packet loss (percent)', 
    'RTT (ping) (ms)', 'Rx instant jitter (ms)', 'Tx instant jitter (ms)',
    'Download (Rx) bandwidth (Mbps)', 'Upload (Tx) bandwidth (Mbps)'
]
target = 'Target'

rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(train_data[qos_features], train_data[target])
importances = pd.DataFrame({'Feature': qos_features, 'Importance': rf_model.feature_importances_})
print("Feature Importances:\n", importances.sort_values(by='Importance', ascending=False))

# Mutual Information
mi_scores = mutual_info_classif(train_data[qos_features], train_data[target])
mi_df = pd.DataFrame({'Feature': qos_features, 'Mutual Information': mi_scores})
print("\nMutual Information Scores:\n", mi_df.sort_values(by='Mutual Information', ascending=False))

# Select features
selected_features = list(set(
    importances[importances['Importance'] > 0.09]['Feature'].tolist() + 
    mi_df[mi_df['Mutual Information'] > 0.05]['Feature'].tolist()
))

print("\nSelected Features for Modeling:\n", selected_features)

# Update datasets with selected features only
train_data = train_data[selected_features + [target]]
val_data = val_data[selected_features + [target]]
test_data = test_data[selected_features + [target]]




### Tx instant jitter (ms) (31.5%) and Tx packet loss (percent) (23.9%) are the most important features based on Random Forest, with strong predictive power.
### Rx instant jitter (ms) (18.6%) and Upload (Tx) bandwidth (Mbps) (9.6%) also show significant contributions.
### Rx instant jitter (ms) (18.2%) and Tx instant jitter (ms) (15.3%) have the strongest dependency with the target, while features like Rx packet loss (percent) contribute almost no information.

### Retained features: Upload (Tx) bandwidth (Mbps), RTT (ping) (ms), Tx packet loss (percent), Rx instant jitter (ms), Tx instant jitter (ms) for further modeling as they show high relevance to the target variable.

## Feature Scaling (Robust Scaler)

### Unlike StandardScaler (uses mean and standard deviation) or Min-Max Scaler (uses min and max values), RobustScaler uses median and interquartile range (IQR) for scaling.
### This makes it less sensitive to outliers, which are common in QoS datasets due to network anomalies.

### It scales the data by centering around the median and adjusting the spread based on the IQR.
### Ensures that non-outlier data is appropriately scaled without being skewed by extreme values.

In [ ]:
from sklearn.preprocessing import RobustScaler
import joblib

# Initialize RobustScaler and fit on training data only
scaler = RobustScaler()
train_scaled = scaler.fit_transform(train_data[selected_features])
val_scaled = scaler.transform(val_data[selected_features])
test_scaled = scaler.transform(test_data[selected_features])

# Convert scaled data back to DataFrame
train_scaled_df = pd.DataFrame(train_scaled, columns=selected_features)
val_scaled_df = pd.DataFrame(val_scaled, columns=selected_features)
test_scaled_df = pd.DataFrame(test_scaled, columns=selected_features)

# Add target variable back
train_scaled_df[target] = train_data[target].reset_index(drop=True)
val_scaled_df[target] = val_data[target].reset_index(drop=True)
test_scaled_df[target] = test_data[target].reset_index(drop=True)

train_scaled_df.to_csv('/kaggle/working/train_scaled_df.csv', index=False)

# Save the scaler
joblib.dump(scaler, '/kaggle/working/robust-scaler.pkl')



## Imbalanced Data Analysis Before Oversampling


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Check class distribution in the training set
class_counts = train_scaled_df[target].value_counts()
print("Class Distribution Before Oversampling in Training Set:\n", class_counts)

# Visualize feature distributions by class for the training set only
for feature in selected_features:
    plt.figure(figsize=(10, 6))
    sns.kdeplot(data=train_scaled_df, x=feature, hue=target, fill=True, alpha=0.7)
    plt.title(f'Feature Distribution by Class: {feature} (Training Set)')
    plt.xlabel(feature)
    plt.ylabel("Density")
    plt.show()



* ### Tx Instant Jitter shows a relatively clearer separation between Good and Poor. Good is skewed toward lower RTT values, while Poor leans toward higher RTT. This feature hence seems crucial for distinguishing Good vs. Poor classes.


* ### Features like Upload (Tx) Bandwidth and Tx Instant Jitter are more likely to be significant in separating the classes.
* ### However, overlap among classes in several features (e.g., Tx Packet Loss & RTT) suggests challenges in class distinction and highlights the need for balancing or using advanced classifiers.
* ### The small density of Fair class implies it may be harder to learn during modeling, emphasizing the need for careful oversampling (like ADASYN) or using class weighting in models.


* ### **These visualizations confirm the need for oversampling techniques (e.g., ADASYN) to balance the Fair class while retaining the distributions of Good and Poor.**
* ### **The separability of features guides feature importance during model evaluation. Features with better class distinction (e.g., RTT, Jitter) should be prioritized in model training.**


## Oversampling with ADASYN

### SMOTE generates synthetic samples for the minority class by interpolating between nearest neighbors, assuming uniform distribution across the class. ADASYN, on the other hand, considers the density of data points within the minority class:

### * It generates more samples for minority class instances that are harder to classify (closer to the decision boundary or sparsely distributed).
### * It generates fewer samples for areas where the minority class is already well-represented.
### * The Fair class is very sparse and likely unevenly distributed across the feature space. ADASYN adapts to this uneven distribution better than SMOTE.
### * The sampling_strategy='minority' parameter in oversampling techniques like ADASYN or SMOTE specifies that the minority class (the class with the fewest samples) will be oversampled to balance the dataset.

In [ ]:
from imblearn.over_sampling import ADASYN

# Separate features and target for oversampling
X_train = train_scaled_df[selected_features]
y_train = train_scaled_df[target]

print("Class Distribution in Training Set Before ADASYN:")
print(y_train.value_counts())


# Adjust sampling_strategy to ensure ADASYN generates synthetic samples
adasyn = ADASYN(sampling_strategy='minority', random_state=42)
X_resampled, y_resampled = adasyn.fit_resample(X_train, y_train)

# Combine resampled data into a DataFrame
train_resampled = pd.DataFrame(X_resampled, columns=selected_features)
train_resampled[target] = y_resampled

# Check the new class distribution
print("Class Distribution After ADASYN:\n", train_resampled[target].value_counts())



## Finalize Datasets for Modeling

In [ ]:
# Save the final training, validation, and testing datasets
train_resampled.to_csv('/kaggle/working/final_train_dataset.csv', index=False)
val_scaled_df.to_csv('/kaggle/working/final_validation_dataset.csv', index=False)
test_scaled_df.to_csv('/kaggle/working/final_test_dataset.csv', index=False)

# Check the structure of the saved datasets
print("Training Dataset:\n", train_resampled.head())
print("Validation Dataset:\n", val_scaled_df.head())
print("Testing Dataset:\n", test_scaled_df.head())


# 🟥 | Model Development

## Dataset Loading

In [ ]:
import pandas as pd

# Load datasets
train_data = pd.read_csv('/kaggle/input/final-datasets/final_train_dataset.csv')
val_data = pd.read_csv('/kaggle/input/final-datasets/final_validation_dataset.csv')
test_data = pd.read_csv('/kaggle/input/final-datasets/final_test_dataset.csv')

print(f"Training set size: {train_data.shape}")
print(f"Validation set size: {val_data.shape}")
print(f"Test set size: {test_data.shape}")


## Baseline Models

### Train simple models to establish baseline performance.

### The baseline model provides a performance reference point for evaluating advanced models.
### It helps determine whether the advanced models improve upon basic methods like Logistic Regression or Decision Tree.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Define baseline models
baseline_models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'SVM': SVC(kernel='rbf', class_weight='balanced', probability=True, random_state=42)
}

# Initialize the dictionary to store trained models
trained_baseline_models = {}

# Train and evaluate baseline models on the validation set
for name, model in baseline_models.items():
    print(f"\nTraining and Evaluating: {name}")
    model.fit(train_data.drop(columns='Target'), train_data['Target'])  # Train the model
    trained_baseline_models[name] = model  # Save the trained model
    
    # Generate predictions
    val_preds = model.predict(val_data.drop(columns='Target'))
    
    # Classification Report
    print(f"{name} Performance on Validation Set:\n")
    print(classification_report(val_data['Target'], val_preds))
    
    # Confusion Matrix
    plt.figure(figsize=(6, 4))
    sns.heatmap(confusion_matrix(val_data['Target'], val_preds), annot=True, fmt='d', cmap='Blues')
    plt.title(f"Confusion Matrix: {name}")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.show()

# Accessing the trained Decision Tree model
dt_model = trained_baseline_models['Decision Tree']
print("Trained Decision Tree model retrieved successfully!")


### Logistic Regression (Overall Accuracy 0.69)

* ### Fair Class has poor performance with a precision of 0.17 and a recall of 0.60.This indicates that when the model predicts "Fair," it is often incorrect, but it captures 60% of actual "Fair" instances.
* ### For Good class, Decent performance with precision and recall around 0.77-0.78.
* ### Slightly better performance than "Fair," but recall (0.60) indicates that the model misses a significant number of "Poor" instances.


### Decision Tree (Overall Accuracy 0.90)


* ### Fair class, Slightly better than Logistic Regression but still poor, with precision of 0.25 and recall of 0.40.
* ### Good class, High performance with both precision and recall around 0.94-0.96.
* ### Strong performance with precision of 0.91 and recall of 0.88 for Poor class.
* ### Macro Average scores for Precision, Recall & F1 between 0.70 & 0.74, much better than Logisitic Regression & SVM.



### Support Vector Machine (Overall Accuracy 0.78)


* ### Best performance on "Fair" compared to other models, with precision of 0.27 and recall of 0.80.
* ### On Poor Class, slightly lower performance than Decision Tree, with precision of 0.91 and recall of 0.67.


### **Decision Tree has the best overall performance with an accuracy of 90%, strong metrics for "Good" and "Poor" classes, and slightly improved handling of the "Fair" class compared to Logistic Regression & SVM. Although the predictions and scores for the "Fair" class is still a challenge.**

## Stacking Ensemble

### Base Models Used : Random Forest, Gradient Boosting, SVM
### Meta Models Used : Logistic Regression.

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

# Define base models
base_models = [
    ('Random Forest', RandomForestClassifier(random_state=42)),
    ('Gradient Boosting', GradientBoostingClassifier(random_state=42)),
    ('SVM', SVC(kernel='rbf', class_weight='balanced', probability=True, random_state=42))
]

# Define meta model
meta_model = LogisticRegression(max_iter=1000, random_state=42)

# Define stacking ensemble
stacking_model = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=5)

# Train stacking model on training set
stacking_model.fit(train_data.drop(columns='Target'), train_data['Target'])

# Evaluate on validation set
val_preds = stacking_model.predict(val_data.drop(columns='Target'))
print("Stacking Ensemble Performance on Validation Set:\n")
print(classification_report(val_data['Target'], val_preds))

# Confusion Matrix
plt.figure(figsize=(6, 4))
sns.heatmap(confusion_matrix(val_data['Target'], val_preds), annot=True, fmt='d', cmap='Blues')
plt.title("Confusion Matrix: Stacking Ensemble")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()


### The stacking ensemble achieved an impressive accuracy of 94% on the validation set, which is significantly better than the baseline models.

### Class-Wise Performance:

### Fair Class (0):
### Precision: 0.43, indicating the model's predictions for "Fair" are often incorrect.
### Recall: 0.60, showing it captures 60% of actual "Fair" instances, better than any baseline model.
### F1-Score: 0.50, demonstrating some improvement in handling this underrepresented class.

### Good Class (1):
### Precision: 0.94, Recall: 0.97, and F1-Score: 0.96, indicating excellent performance for the majority class.
### Poor Class (2):
### Precision: 1.00, Recall: 0.93, and F1-Score: 0.97, suggesting outstanding performance, though it misses a few instances.

### Macro Average : Precision: 0.79, Recall: 0.83, and F1-Score: 0.81, indicating improved balance compared to baseline models. Macro Average provides a much more realistic view than Weighted Average which takes into account label ratios of the dataset.

### **The stacking ensemble outperforms all baseline models and serves as a robust solution for this dataset.**
### **While the ensemble performs exceptionally well on the "Good" and "Poor" classes, additional attention is needed for the "Fair" class.**

# 🔧| Hyperparameter Tuning

## Setup for Base Model Tuning

### This step involves using GridSearchCV and RandomizedSearchCV with Stratified K-Fold Cross-Validation to optimize the hyperparameters of the base models (Random Forest, Gradient Boosting, and SVM). The goal is to identify the best-performing parameter combinations while ensuring the evaluation is unbiased across stratified folds of the training data.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score

# Define parameter grids for base models
rf_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

gb_param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 10]
}

svm_param_grid = {
    'C': [0.1, 1, 10],
    'gamma': [0.001, 0.01, 0.1],
    'kernel': ['rbf', 'linear']
}

# Initialize models
rf_model = RandomForestClassifier(random_state=42)
gb_model = GradientBoostingClassifier(random_state=42)
svm_model = SVC(probability=True, random_state=42)

# Define cross-validation strategy
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


## Randomized Search for Base Models


### This code performs hyperparameter tuning for the base models using RandomizedSearchCV (for Random Forest and Gradient Boosting) and GridSearchCV (for SVM). 

### It identifies the best hyperparameter combinations by evaluating performance across Stratified K-Fold Cross-Validation on the training set, ensuring robust model optimization while mitigating overfitting risks.

In [ ]:
# Randomized Search for Random Forest
rf_random_search = RandomizedSearchCV(
    estimator=rf_model,
    param_distributions=rf_param_grid,
    n_iter=20,
    scoring='accuracy',
    cv=cv,
    verbose=2,
    random_state=42,
    n_jobs=-1
)
rf_random_search.fit(train_scaled_df[selected_features], train_scaled_df['Target'])
print("Best Random Forest Parameters:", rf_random_search.best_params_)

# Randomized Search for Gradient Boosting
gb_random_search = RandomizedSearchCV(
    estimator=gb_model,
    param_distributions=gb_param_grid,
    n_iter=20,
    scoring='accuracy',
    cv=cv,
    verbose=2,
    random_state=42,
    n_jobs=-1
)
gb_random_search.fit(train_scaled_df[selected_features], train_scaled_df['Target'])
print("Best Gradient Boosting Parameters:", gb_random_search.best_params_)

# Grid Search for SVM
svm_grid_search = GridSearchCV(
    estimator=svm_model,
    param_grid=svm_param_grid,
    scoring='accuracy',
    cv=cv,
    verbose=2,
    n_jobs=-1
)
svm_grid_search.fit(train_scaled_df[selected_features], train_scaled_df['Target'])
print("Best SVM Parameters:", svm_grid_search.best_params_)


### Best Random Forest Parameters: {'n_estimators': 300, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 30}
### Best Gradient Boosting Parameters: {'n_estimators': 300, 'max_depth': 3, 'learning_rate': 0.1}
### Best SVM Parameters: {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}


## Meta Model Tuning

### Here, we perform hyperparameter tuning for the Logistic Regression meta model using GridSearchCV. 
### The process evaluates combinations of regularization strength (C), penalty (l2), and solver (lbfgs) across Stratified K-Fold Cross-Validation on the training set.

In [ ]:
# Define parameter grid for Logistic Regression
lr_param_grid = {
    'C': [0.1, 1, 10],
    'penalty': ['l2'],
    'solver': ['lbfgs']
}

# Logistic Regression Grid Search
lr_grid_search = GridSearchCV(
    estimator=LogisticRegression(random_state=42, max_iter=1000),
    param_grid=lr_param_grid,
    scoring='accuracy',
    cv=cv,
    verbose=2,
    n_jobs=-1
)
lr_grid_search.fit(train_scaled_df[selected_features], train_scaled_df['Target'])
print("Best Logistic Regression Parameters:", lr_grid_search.best_params_)


### Best Logistic Regression Parameters: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}


## Build Tuned Stacking Ensemble

### A stacking ensemble is created using the best-tuned estimators (Random Forest, Gradient Boosting, and SVM) and a Logistic Regression model as the meta-model.

### The tuned stacking model is trained on the training dataset (train_scaled_df) with the selected features.

In [ ]:
from sklearn.ensemble import StackingClassifier

# Define tuned base models and meta model
base_models_tuned = [
    ('Random Forest', rf_random_search.best_estimator_),
    ('Gradient Boosting', gb_random_search.best_estimator_),
    ('SVM', svm_grid_search.best_estimator_)
]
meta_model_tuned = lr_grid_search.best_estimator_

# Define stacking ensemble
stacking_model_tuned = StackingClassifier(
    estimators=base_models_tuned,
    final_estimator=meta_model_tuned,
    cv=cv
)

# Train stacking ensemble
stacking_model_tuned.fit(train_scaled_df[selected_features], train_scaled_df['Target'])


print("UnTuned Stacking Ensemble Performance on Validation Set:\n")
print(classification_report(val_scaled_df['Target'], val_preds))

plt.figure(figsize=(6, 4))
sns.heatmap(confusion_matrix(val_data['Target'], val_preds), annot=True, fmt='d', cmap='Blues')
plt.title("Confusion Matrix: Stacking Ensemble Untuned")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

# Evaluate stacking ensemble on validation set
y_pred_stack = stacking_model_tuned.predict(val_scaled_df[selected_features])
print("Tuned Stacking Ensemble Performance on Validation Set:\n")
print(classification_report(val_scaled_df['Target'], y_pred_stack))

plt.figure(figsize=(6, 4))
sns.heatmap(confusion_matrix(val_data['Target'], y_pred_stack), annot=True, fmt='d', cmap='Blues')
plt.title("Confusion Matrix: Stacking Ensemble Tuned")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()



### **Untuned Stacking Ensemble Performance**

### -   Accuracy: 94%
### -   Macro Average F1-Score: 0.81
### -   The untuned ensemble already performs well but shows some inconsistency in precision and recall for the minority class ("Fair")

### **Tuned Stacking Ensemble Performance**

### -   Accuracy: 96%
### -   Macro Average F1-Score: 0.87
### -   Precision for the "Fair" class improved significantly (from 43% to 75%).
### -   The f1 score  for the "Fair" class improved (from 50% to 67%), indicating better handling of the minority class.
### -   Slight improvements in "Good" and "Poor" class predictions, maintaining high precision and recall.

### **Fine-tuning the base and meta models optimized the ensemble's ability to generalize on unseen validation data.**
### **Better f1 score for the "Fair" class demonstrates that the tuned model is less biased towards majority classes.**

# 🤖| Explainable AI

 ## SHAP for Random Forest




* ### The summary plot shows the global importance of each feature for the Random Forest model.
* ### The features are ranked from top to bottom based on their average contribution to the predictions across all validation samples.
* ### The mean(|SHAP value|) on the x-axis indicates the average magnitude of a feature's influence on the model's output.



### Summary Plot

In [ ]:
import shap
import matplotlib.pyplot as plt

for idx, class_label in enumerate(rf_random_search.best_estimator_.classes_):
    print(f"Class Index: {idx}, Class Label: {class_label}")

# Initialize SHAP explainer for Random Forest
rf_explainer = shap.TreeExplainer(rf_random_search.best_estimator_)

# Generate SHAP values for the validation set
rf_shap_values = rf_explainer.shap_values(val_scaled_df[selected_features])

# Global Feature Importance for Random Forest
shap.summary_plot(rf_shap_values, val_scaled_df[selected_features], feature_names=selected_features)


### Global Feature Importance for Random Forest
### Top Influential Features:

### -   Tx instant jitter (ms) has the highest average impact on model predictions for all three classes (Class 0, Class 1, Class 2).
### -   Tx packet loss (percent) and **Rx instant jitter (ms)** also significantly affect predictions.

### **The inference is on improving the key QoS parameters, such as reducing jitter and packet loss, as they have the most significant impact on predicting the quality of the video conferencing service.**

### Force Plots

* ### The force plot shows the explanation for a specific prediction (here at instance index 10).
* ### It highlights how individual features contribute to pushing the prediction away from the base value (model's expected output without any feature input).

In [ ]:
import shap
shap.initjs()  # Initialize JavaScript for SHAP visualizations

for idx, class_label in enumerate(rf_random_search.best_estimator_.classes_):
    print(f"Class Index: {idx}, Class Label: {class_label}")

# Local Explanation for a Single Instance
instance_idx = 10  # Example instance index

# Extract predicted class and probabilities
predicted_class = rf_random_search.best_estimator_.predict(val_scaled_df[selected_features])[instance_idx]
predicted_probabilities = rf_random_search.best_estimator_.predict_proba(val_scaled_df[selected_features])[instance_idx]

print(f"Predicted Class: {predicted_class}")
print(f"Predicted Probabilities: {predicted_probabilities}")

# Render force plots for each class
for class_idx, class_name in enumerate(['Class 0', 'Class 1', 'Class 2']):  # Adjust class names
    print(f"\nForce Plot for {class_name} (Class Index: {class_idx}):")
    shap.force_plot(
        rf_explainer.expected_value[class_idx],  # Base value for the class
        rf_shap_values[class_idx][instance_idx],  # SHAP values for the instance and class
        val_scaled_df.iloc[instance_idx][selected_features], matplotlib=True  # Feature values for the instance
    )




### Force Plot at instance - index 10
### Interpretation:

### -   Base Value: The model's average output value for all instances in the dataset.
### -   f(x): The predicted probability (or log-odds) for the given instance.
### -   Red Arrows:
###     -   Features with **positive contributions** that push the prediction towards the predicted class.
###     -   Example: **Rx instant jitter (ms)** and **Tx instant jitter (ms)** are pushing the prediction strongly in the direction of the predicted class.
### -   Blue Arrows:
###     -   Features with negative contributions that oppose the prediction.

### Dependence Plot 

### The SHAP Dependence Plot visualizes the relationship between a specific feature (e.g., Tx packet loss (percent)) and its contribution to predictions for a given class (e.g., Poor).


In [ ]:
for idx, class_label in enumerate(rf_random_search.best_estimator_.classes_):
    print(f"Class Index: {idx}, Class Label: {class_label}")

def check_shap_class(idx, shap_values, class_labels):
    print(f"SHAP values correspond to Class: {class_labels[idx]}")
check_shap_class(2, rf_shap_values, rf_random_search.best_estimator_.classes_)


shap.dependence_plot(
    'Tx packet loss (percent)',
    rf_shap_values[2],  # Class-specific SHAP values for class Poor (index 2 for other classes)
    val_scaled_df[selected_features]
)




###    **Feature on the X-axis**:

###       The x-axis represents the values of the feature `Tx packet loss (percent)`. This is the input feature for which the relationship with the model's output (SHAP value) is being analyzed.-   

### **SHAP Value on the Y-axis**:

###       The y-axis represents the SHAP values for the feature `Tx packet loss (percent)`. A positive SHAP value indicates that the feature pushes the prediction towards the respective class, while a negative value pushes it away.-   

### **Color Coding (Additional Feature)**:

###      The color of the points represents the value of another feature, `Tx instant jitter (ms)`, to show interaction effects.
###      Higher `Tx instant jitter (ms)` values (darker pink) might amplify the impact of `Tx packet loss (percent)` on the prediction.-   

### **Insights from the Plot**:


* ###       When `Tx packet loss (percent)` is near `-1`, the SHAP value is mostly negative, meaning this feature has a negative impact on the prediction for the specific class (`Poor`).
* ###       As `Tx packet loss (percent)` increases from 0, the SHAP values become more positive, indicating that higher packet loss increases the likelihood of predicting the class (`Poor`).
* ###       The interaction with `Tx instant jitter (ms)` is visible through the color variations. For example, at the same `Tx packet loss (percent)` value, points with higher `Tx instant jitter (ms)` (pink) have different SHAP values than those with lower jitter (blue).


### Decision Plot

### The SHAP Decision Plot illustrates how SHAP values for all features cumulatively impact the prediction across all instances for each class (e.g., Fair, Good, Poor).


In [ ]:
# Generate SHAP Decision Plots for all instances for each class
for class_idx, class_label in enumerate(['Fair', 'Good', 'Poor']):
    print(f"SHAP Decision Plot for Class {class_label} (Class Index: {class_idx}):")
    shap.decision_plot(
        rf_explainer.expected_value[class_idx],  # Base value for the class
        rf_shap_values[class_idx],  # SHAP values for all instances for the class
        feature_names=selected_features,
        title=f"Decision Plot for Class {class_label} (All Instances)",
        show=True
    )

### Model Output Value:


* ### The x-axis represents how much each feature contributes to the final output for a specific class.
* ### Values closer to 1 indicate a higher likelihood of being classified as the respective class, while negative or closer to 0 suggests a lower likelihood.
* ### Color Bar: Indicates the feature value (blue for lower values, pink for higher values).


### **Class Fair (Class Index: 0)**

###   The plot shows the contribution of each feature to the model's output value for the "Fair" class across all instances.

###       **Rx instant jitter (ms)** and **Tx packet loss (percent)** appear to have the most significant impact in pushing the model output value toward or away from classifying as "Fair."
###       Negative contributions (left of 0) reduce the likelihood of being classified as "Fair," while positive contributions (right of 0) increase it.

### **Class Good (Class Index: 1)**

###   This plot represents contributions to the "Good" class.

###       Features like **Tx instant jitter (ms)** and **Tx packet loss (percent)** strongly influence predictions for "Good."
###       Some instances have **Tx instant jitter (ms)** contributing positively, moving the model's output value toward the "Good" class, while others push it away.

### **Class Poor (Class Index: 2)**

###   This plot shows contributions to the "Poor" class.

###       **Tx instant jitter (ms)** and **Tx packet loss (percent)** are significant factors for predicting "Poor."
###       Many instances are influenced heavily by these features, pushing the output value positively (toward classifying as "Poor").



### **Insights**

### **Feature Importance**:

###   Across all classes, **Tx instant jitter (ms)** and **Tx packet loss (percent)** play a significant role in shaping predictions.
###   The direction and magnitude of influence vary based on the class.

### **Divergent Trends**:

###   The same feature can have opposing effects across different classes, highlighting its nuanced role in decision-making.

##  LIME for SVM


* ### LIME explains the predictions for an individual instance by generating perturbations (slight variations) of the instance and observing how the model's predictions change.
* ### To simulate the data distribution around the instance, LIME uses the training data as a reference to build its local neighborhood.
* ### This ensures that the perturbations are realistic and consistent with the distribution the model was trained on.


### Initialize LIME Tabular Explainer

### Initialize LIME Explainer: The LimeTabularExplainer is initialized with the training data, feature names, and class names to provide explanations for classification predictions.
### Classification Mode: The mode='classification' parameter ensures the explainer is configured for classification tasks, mapping features to the class probabilities.

In [ ]:
import lime
from lime.lime_tabular import LimeTabularExplainer
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Initialize the LIME explainer
lime_explainer = LimeTabularExplainer(
    training_data=train_scaled_df[selected_features].values,  # Use training data
    training_labels=train_scaled_df['Target'].values,
    feature_names=selected_features,
    class_names=['Fair', 'Good', 'Poor'],  # Provide the class names
    mode='classification'
)



### Explain an Instance & Visualization

### explain_instance explains the prediction for a specific validation instance using the predict_proba method of the tuned SVM, highlighting the top 5 contributing features.

In [ ]:
# Choose a specific instance index from the validation set
instance_idx = 40  # Example instance index
instance = val_scaled_df[selected_features].iloc[instance_idx].values.reshape(1, -1)

# Generate an explanation for the instance
lime_explanation = lime_explainer.explain_instance(
    data_row=instance.flatten(),  # Flatten the instance for input
    predict_fn=svm_grid_search.best_estimator_.predict_proba,  # Use the predict_proba method of the tuned SVM
    num_features=5  # Limit to the top 5 contributing features
)

# Display the explanation as a table
lime_explanation.show_in_notebook()

# Save the explanation as an HTML file
lime_explanation.save_to_file('/kaggle/working/lime_explanation.html')

# Visualize the explanation as a bar plot
lime_explanation.as_pyplot_figure()
plt.title(f'LIME Explanation for Instance {instance_idx}')
plt.show()


### Prediction Summary (Top Left)


* ### Fair (0.00), Good (0.94), Poor (0.06)
* ### The model predicts "Poor" with the highest probability (88%).

### Explanation of the Prediction (Bottom Bar Plot)

### This bar plot shows the top 5 features and their impact (positive or negative) on the predicted class:
### The low value of Tx Instant Jitter contributes significantly to the instance being classified as Good.
### The high RTT (ping) value pulls the prediction slightly away from Good, indicating it's less favorable for this class.


### **The strongest contributor is Tx Instant Jitter (ms) ≤ -0.51, which heavily supports the classification of Good.**
### **Despite negative influences like RTT (ping) (ms) > 0.47 and Tx Packet Loss (percent) > 0.45, the positive contributions outweigh them, leading to a confident classification.**
### **The top green bar (Tx Instant Jitter (ms) ≤ -0.51) visually dominates, showing it is the primary reason for the Good classification.**

# ✔️| Final Model Evaluation

## Evaluating Performance on the Test Set

### Generate Predictions on Test Set

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, precision_recall_curve, log_loss
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Generate predictions
y_test = test_scaled_df['Target']
X_test = test_scaled_df[selected_features]

y_pred_test = stacking_model_tuned.predict(X_test)
y_pred_proba_test = stacking_model_tuned.predict_proba(X_test)

# Display predictions
print(f"Predicted Labels: {y_pred_test[:10]}")
print(f"Predicted Probabilities (first 10 rows):\n{y_pred_proba_test[:10]}")


### Calculate Performance Metrics & Confusion Matrix

### Here, we calculate and display the classification report with precision, recall, F1-score, and support for each class along with a Confusion Matrix.
### Also Log Loss & Macro ROC-AUC is calculated.

* ### Log Loss focuses on the calibration of probabilities. It rewards well-calibrated predictions and penalizes incorrect predictions with high confidence.
* ### ROC-AUC evaluates the model's discrimination ability — its ability to rank true positives higher than false positives, regardless of probability calibration.


In [ ]:
# Classification report
print("Classification Report on Test Data:")
print(classification_report(y_test, y_pred_test, target_names=['Fair', 'Good', 'Poor']))

# Log Loss
logloss = log_loss(y_test, y_pred_proba_test)
print(f"Log Loss: {logloss}")

# ROC-AUC (macro and per class)
roc_auc_macro = roc_auc_score(y_test, y_pred_proba_test, multi_class='ovr', average='macro')
print(f"Macro ROC-AUC: {roc_auc_macro}")

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred_test)

# Heatmap Visualization
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Fair', 'Good', 'Poor'], yticklabels=['Fair', 'Good', 'Poor'])
plt.title("Confusion Matrix: Test Data")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()


* ### Fair: Precision and recall are 0.83, indicating balanced performance for this class, though based on fewer instances (support: 6).
* ### Good: Precision and recall are 0.96, showing excellent performance with the highest support (69 instances).
* ### Poor: Precision and recall are 0.97, indicating strong performance across all metrics with support of 59 instances.


### Log Loss: Low value (0.1859), indicating that the model generates well-calibrated probabilities.

### Macro ROC-AUC: 0.98, showing excellent discriminatory ability for all classes in a one-vs-all setup.

### ROC Curves


* ### The ROC curve is a graphical representation of a classifier's ability to distinguish between classes.
* ### It plots the True Positive Rate (TPR) (Sensitivity) against the False Positive Rate (FPR) at different thresholds for classification.


---- 


* ### The AUC measures the overall performance of the classifier.
* ### An AUC of 1.0 indicates perfect classification, while 0.5 indicates no discrimination (equivalent to random guessing).


-----


* ### In multi-class problems, the ROC curve is computed for each class, treating that class as "positive" and all others as "negative."
* ### This process is repeated for each class, generating a separate ROC curve for each.


In [ ]:
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Binarize the target labels
y_test_binary = label_binarize(y_test, classes=['Fair', 'Good', 'Poor'])

# Generate ROC Curves
plt.figure(figsize=(10, 8))

for class_idx, class_label in enumerate(['Fair', 'Good', 'Poor']):
    # Compute False Positive Rate, True Positive Rate, and thresholds
    fpr, tpr, _ = roc_curve(y_test_binary[:, class_idx], y_pred_proba_test[:, class_idx])
    # Compute Area Under Curve (AUC)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f"ROC Curve for {class_label} (AUC = {roc_auc:.2f})")

# Plot the random guess line
plt.plot([0, 1], [0, 1], 'k--', label="Random Guess")
plt.title("ROC Curves (One-vs-All)")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.show()



* ### A steep curve closer to the top-left corner indicates strong performance, meaning the model achieves high sensitivity with low false positives.
* ### Each class has an AUC (Area Under the Curve) of 0.99, which indicates near-perfect discrimination capability. The closer the AUC is to 1.0, the better the model is at separating that class from the rest.
* ### The dashed diagonal line represents a random classifier (AUC = 0.5). Any ROC curve above this line reflects a model better than random guessing.


### Precision-Recall Curves

### Precision-Recall (PR) curves evaluate the trade-off between precision (positive predictive value) and recall (sensitivity) at different probability thresholds for each class (Fair, Good, Poor).

-----

### Here, we :

* ### Compute precision and recall for each threshold using the binarized true labels (y_test_binary) and predicted probabilities (y_pred_proba_test) for each class.

* ### Calculates the Average Precision (AP), a summary statistic that represents the area under the Precision-Recall curve. Higher AP values indicate better performance.


In [ ]:
from sklearn.metrics import precision_recall_curve, average_precision_score

# Generate Precision-Recall Curves
plt.figure(figsize=(10, 8))

for class_idx, class_label in enumerate(['Fair', 'Good', 'Poor']):
    # Compute Precision, Recall, and thresholds
    precision, recall, _ = precision_recall_curve(y_test_binary[:, class_idx], y_pred_proba_test[:, class_idx])
    # Compute Average Precision Score (AP)
    avg_precision = average_precision_score(y_test_binary[:, class_idx], y_pred_proba_test[:, class_idx])
    plt.plot(recall, precision, label=f"Precision-Recall for {class_label} (AP = {avg_precision:.2f})")

plt.title("Precision-Recall Curves (One-vs-All)")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.legend()
plt.show()



* ### Precision and recall drop more sharply for the Fair class compared to the other two, likely because there are fewer instances of Fair in the dataset (imbalanced class).
* ### Average Precision (AP) for Fair is 0.88, indicating relatively good but not perfect performance.
* ### Both Good & Poor classes maintain high precision and recall for most thresholds, resulting in AP values close to 0.99. This suggests the model is very effective at predicting these classes.


# 🚅| Save Model & Pipeline 

## Save the Model

In [ ]:
import joblib

# Save the tuned stacking ensemble model
model_path = '/kaggle/working/stacking_model_tuned.pkl'
joblib.dump(stacking_model_tuned, model_path)
print(f"Tuned Stacking Ensemble Model saved at {model_path}")

# Reload the saved model
loaded_model = joblib.load(model_path)

# Check if the loaded model is the same as the trained one
print(loaded_model)



## Save the Preprocessing Pipeline

In [ ]:
# Save the RobustScaler used for preprocessing
scaler_path = '/kaggle/working/robust_scaler.pkl'
joblib.dump(scaler, scaler_path)
print(f"Scaler saved at {scaler_path}")
